In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numpy_financial as npf
import pandas as pd
from datetime import datetime as dt
import warnings
warnings.filterwarnings('ignore')
import math
from dateutil.relativedelta import relativedelta
import os
from scipy.stats import norm

In [2]:
data_folder_name = 'Data'
instruments_folder_name = "Instruments"

In [3]:
def return_conversion (r,n,this_period = 1):
    return (1+r)**(n/this_period) - 1

In [4]:
def to_currency (value, multiplier=100, add_currency_symbol=True, currency_symbol = '₹',rounding=0):
    negative_sign = ""
    if value < 0:
        negative_sign = "-"
        value = abs(value)
    value = round(value,rounding)
    decimal_value = round(value%1,rounding)
    value = int(round((value-decimal_value),0))
    if decimal_value == 0:
        currency_string = "" 
    else:
        currency_string = "." + str(decimal_value)[2:]
    base = 1_000
    set = value%base
    currency_string = str(int(round(set,0))) + currency_string
    if (len(str(set)) < len(str(value))) & (len(str(set)) < len(str(base))-1):
        zeros = "".join(['0' for _ in range(len(str(base))-1-len(str(set)))])
        currency_string = zeros + currency_string
        
    converted_value = set
    if set == value:
        if add_currency_symbol:
            return currency_symbol + " " + negative_sign + currency_string
        else: 
            return negative_sign + currency_string
    else:
        while converted_value != value:
            base = base * multiplier
            set = int(round((value%base - converted_value)/base * multiplier,0))
            currency_string = str(int(set)) + "," + currency_string
            if (len(str(int(round(set*base/multiplier,0)))) < len(str(value))) & (len(str(set)) < len(str(multiplier))-1):
                zeros = "".join(['0' for _ in range(len(str(multiplier))-1-len(str(set)))])
                currency_string = zeros + currency_string
            converted_value = int(round(converted_value + (set*base/multiplier),0))

    if add_currency_symbol:
        return currency_symbol + " " + negative_sign + currency_string
    else: 
        return negative_sign + currency_string

to_currency(-78361.78264643789, multiplier=100,rounding=0, add_currency_symbol=True)

'₹ -78,362'

In [9]:
class Instrument:
    def __init__(self,name,type,ticker=None,r_annual=0,
        entry_transaction_cost=0, exit_transaction_cost=0):
        self.name = name
        self.ticker = ticker
        self.type = type
        self.historical = pd.DataFrame()
        self.r_annual = r_annual
        self.entry_transaction_cost = entry_transaction_cost
        self.exit_transaction_cost = exit_transaction_cost
    
    def __repr__ (self):
        return f'''{self.name}'''
    def __str__ (self):
        return self.__repr__()
    
    def fetch_historical(self,folder_path,file_name):
        self.historical = pd.read_csv(os.path.join(folder_path, file_name))
        self.historical['Date'] = pd.to_datetime(self.historical['Date'])
        self.historical.sort_values(by='Date', inplace=True)
        self.historical['change'] = (self.historical['Close'] \
            - self.historical['Close'].shift(periods=1)) \
            / self.historical['Close'].shift(periods=1)
    
    def get_value(self, date):
        return self.historical[self.historical['Date']<=date].iloc[-1]['Close']
    
    
    def get_fall_prob (self, fall,date, lookback_years = None):
        if self.type == 'debt':
            if self.r_annual >= fall:
                return 1
            else: 
                return 0
        if not lookback_years:
            lookback_years = 10
        dist_df = self.historical[self.historical['Date']<=date]\
            .iloc[-(lookback_years*250):]
        dist_prob = dist_df[dist_df['change']<=fall].shape[0] / dist_df.shape[0]
        norm_prob = norm(dist_df['change'].mean(), dist_df['change'].std()).cdf(fall)
        return dist_prob if dist_prob > norm_prob else norm_prob

    def get_return (self,date, lookback_years = None):
        if self.type == 'equity':
            if not lookback_years:
                lookback_years = 10
            return self.historical[self.historical['Date']<=date]\
                .iloc[-(lookback_years*250):]['change'].mean()
        else:
            return return_conversion(self.r_annual,1,365)
    
    def future_exit_value (self, entry_date, exit_date, investment_amount):
        r = self.get_return(entry_date)
        n_days = (exit_date-entry_date).days
        holding_period_years = n_days/365
        fv = (investment_amount-self.entry_transaction_cost) * (1+r)**n_days - self.exit_transaction_cost
        pnl = fv-investment_amount
        tax_liability = 0
        if pnl > 0:
            if self.type == 'equity':
                if holding_period_years >=1: #Long Term
                    tax_liability = pnl * .1
                else: #Short Term
                    tax_liability = pnl * .15
            elif self.type == 'debt':
                if holding_period_years >=3: #long Term
                    tax_liability = pnl * .2
                else: #short term
                    tax_liability = pnl * .3

        future_exit_value = fv - tax_liability
        return future_exit_value

rf = Instrument('rf',type='debt',r_annual=.06, entry_transaction_cost=10,exit_transaction_cost=20)
cash_instrument = Instrument('cash',type='debt',r_annual=0, entry_transaction_cost=0,exit_transaction_cost=0)

In [6]:
class Position:
    def __init__(self, instrument, date,investment_amount):
        self.instrument = instrument
        self.entry_date = date
        self.position_name = str(self.instrument) + "_on_" + self.entry_date.strftime('%d%b%y')
        self.pnl = 0
        self.current_value = investment_amount-instrument.entry_transaction_cost
        if self.instrument.type == 'equity':
            self.number_of_units = self.current_value\
                    /self.instrument.get_value(date)
        else: 
            self.number_of_units = 1
        self.entry_value = self.current_value
        self.exit_value = self.current_value - instrument.exit_transaction_cost
        self.tax_liability = 0
        self.last_update = date
        self.is_active = True
    
    def __repr__ (self):
        return f'''{self.position_name} {to_currency(self.current_value)} | exit: {to_currency(self.exit_value)}, entry: {to_currency(self.entry_value)} | update:{self.last_update.strftime('%d%b%y')}'''
    def __str__ (self):
        return self.__repr__()
    def __float__(self):
        return float(self.exit_value)
    
    def update_current_value (self,date):
        if self.instrument.type == 'equity':
            self.current_value = self.instrument.get_value(date) * self.number_of_units
        if self.instrument.type == 'cash':
            pass
        if self.instrument.type == 'debt':
            self.current_value = self.entry_value * (1+return_conversion(self.instrument.r_annual,1,365))**(date-self.entry_date).days
            
    
    def update(self,date):
        self.update_current_value(date)
        self.pnl = self.current_value - self.instrument.exit_transaction_cost - self.entry_value
    
        #Computing tax Component
        holding_period_years = relativedelta(date,self.entry_date).years
        if self.pnl > 0:
            if self.instrument.type == 'equity':
                if holding_period_years >=1: #Long Term
                    self.tax_liability = self.pnl * .1
                else: #Short Term
                    self.tax_liability = self.pnl * .15
            elif self.instrument.type == 'debt':
                if holding_period_years >=3: #long Term
                    self.tax_liability = self.pnl * .2
                else: #short term
                    self.tax_liability = self.pnl * .3
        else:
            self.tax_liability = 0
        self.exit_value = self.current_value - self.tax_liability - self.instrument.exit_transaction_cost
        self.last_update = date

    def future_exit_value(self, date, exit_date):
        r = self.instrument.get_return(date)
        n_days = (exit_date-self.entry_date).days
        holding_period_years = n_days/365
        fv = (self.current_value) * (1+r)**n_days - self.instrument.exit_transaction_cost
        pnl = fv-self.entry_value
        tax_liability = 0
        if pnl > 0:
            if self.type == 'equity':
                if holding_period_years >=1: #Long Term
                    tax_liability = pnl * .1
                else: #Short Term
                    tax_liability = pnl * .15
            elif self.type == 'debt':
                if holding_period_years >=3: #long Term
                    tax_liability = pnl * .2
                else: #short term
                    tax_liability = pnl * .3

        future_exit_value = fv - tax_liability
        return future_exit_value
    
    def withdraw(self,date,amount):
        self.update(date)
        if amount <= self.exit_value:
            self.current_value = self.current_value - amount
            self.exit_value = self.current_value - self.instrument.exit_transaction_cost
            self.last_update = date
            return amount
        else:
            return 0

    def withdraw_full(self,date):
        self.is_active = False
        return self.withdraw(date=date,amount=self.exit_value)

# class Cash (Position):
#     def __init__(self,date,investment_amount):
#         cash = Instrument('Cash',type='cash',entry_transaction_cost=0,exit_transaction_cost=0)
#         super().__init__(cash, date,investment_amount)
#         self.position_name = str(self.instrument)
    
#     def __repr__ (self):
#         return f'''{self.position_name} {to_currency(self.current_value)}'''
#     def __str__ (self):
#         return self.__repr__()
#     def __float__(self):
#         return float(self.current_value - self.instrument.entry_transaction_cost)

#     def update(self,date):
#         self.last_update = date
    
#     def deposit(self,date, amount):
#         self.current_value = self.current_value + amount - self.instrument.entry_transaction_cost
#         self.last_update = date
        




            
        




In [7]:
class Goal:
    def __init__ (self,name, maturity_date,inception_date,
        confidence,maturity_value,min_maturity_value,pmt=0):
        self.name = name
        self.maturity_date = maturity_date
        self.inception_date = inception_date
        self.pmt = pmt
        self.confidence = confidence
        self.payday = 1
        self.maturity_value = maturity_value
        self.min_maturity_value = min_maturity_value
        self.cashflows = pd.DataFrame(index=pd.date_range(start=inception_date, end=maturity_date))\
            .reset_index().rename(columns={'index':'date'})
        self.cashflows['downpayment'] = 0
        self.cashflows['spend'] = 0
        self.cashflows['emi'] = 0
        self.cashflows.loc[self.cashflows['date']==inception_date,'downpayment'] = pmt
        self.cashflows.loc[self.cashflows['date']==maturity_date,'spend'] = maturity_value
        self.emi = self.calculate_emi(rf=return_conversion(.06,1,250))
        self.positions = []
        self.current_value = 0
        self.update(inception_date)

    def __repr__ (self):
        return f'''{self.name} | {to_currency(self.maturity_value)} on {self.maturity_date.strftime('%d %b %Y')} | EMI: {to_currency(self.emi)}, Downpayment: {to_currency(self.pmt)}'''
    def __str__ (self):
        return self.__repr__()
        
    def calculate_emi(self,rf):
        tolerance = 1
        emi_guess = npf.pmt(rate = rf, nper=(self.maturity_date-self.inception_date).days/30, 
                        fv=-1*self.maturity_value, pv=self.pmt, when='begin')
        surplus = 0

        while not((surplus>0)&(surplus<tolerance)):
            emi_guess = emi_guess - surplus/((self.maturity_date-self.inception_date).days/30)
            self.cashflows.loc[self.cashflows['date'].dt.day==self.payday,'emi'] = emi_guess
            self.cashflows['cashflow'] = self.cashflows['downpayment'] + self.cashflows['emi'] - self.cashflows['spend']
            surplus = npf.npv(rf,self.cashflows['cashflow'])
            
        self.cashflows.loc[self.cashflows['date'].dt.day==self.payday,'emi'] = emi_guess
        self.cashflows['cashflow'] = self.cashflows['downpayment'] + self.cashflows['emi'] - self.cashflows['spend']
        return emi_guess

    def update_emi(self,date,updated_emi):
        self.cashflows.loc[(self.cashflows['date'].dt.day==self.payday)&(self.cashflows['date']>=date),'emi'] = updated_emi
        self.emi = updated_emi
    def collect_cash(self,date):
        available_cash = \
            self.cashflows.loc[self.cashflows['date']==date,'downpayment'] \
                + self.cashflows.loc[self.cashflows['date']==date,'emi']
        if available_cash > 0:
            cash_position = Position(instrument=cash_instrument,date=date,investment_amount=available_cash)
            self.positions.append(cash_position)
        return available_cash
            
    
    def update(self,date):
        self.collect_cash(date)
        self.current_value = 0
        for position in self.positions:
            position.update()
            self.current_value += position.exit_value
        return self.current_value

    def filter_instruments(self,date,instruments_list):
        self.cashflows
        t_plus_1_pv_future_cashflows = 
        
        return applicable_instruments_list
    
    def best_instrument(self,date, instruments_list):
        best_fv = 0
        best_instrument = None
        for instrument in instruments_list:
            instrument_fv = instrument.future_exit_value(investment_amount=self.current_value,
                entry_date=date,exit_date=self.maturity_date)
            if instrument_fv >= best_fv:
                best_fv = instrument_fv
                best_instrument = instrument
        return best_instrument

    def switch_positions(self,date,best_instrument):
        for position in self.positions:
            if position.is_active:
                current_future_exit_value = position.future_exit_value(date=date,exit_date=self.maturity_date)
                best_instrument_future_exit_value = \
                    best_instrument.future_exit_value(investment_amount=position.exit_value,
                        entry_date=date,exit_date=self.maturity_date)
            if best_instrument_future_exit_value > current_future_exit_value:
                transfer_amount = position.withdraw_full(date)
                new_position = Position(instrument=best_instrument,date=date,investment_amount=transfer_amount)
                self.positions.append(new_position)

    def calibrate(self,date, instruments_list):
        self.update(date)
        best_instrument = self.best_instrument(date=date, instruments_list = instruments_list)
        self.switch_position(date,best_instrument)
                
                


In [8]:
a=0
date_lists = [dt(2015,2,1),dt(2015,7,3),dt(2017,2,4),dt(2022,2,7)]

nifty = Instrument('Nifty',type='equity',ticker='N50',entry_transaction_cost=20, exit_transaction_cost=20)
nifty.fetch_historical(folder_path=os.path.join(data_folder_name, instruments_folder_name),file_name='NIFTY50_Data.csv')
rf = Instrument('rf',type='debt',r_annual=.06, entry_transaction_cost=10,exit_transaction_cost=20)
print(nifty,rf)
nifty_p = Position(instrument=nifty,date=date_lists[0],investment_amount=10_000)
rf_p = Position(instrument=rf,date=date_lists[0],investment_amount=10_000)
cash_p = Position(instrument=cash_instrument,date=date_lists[0],investment_amount=10_000)
print(f'''{date_lists[0].strftime("%d %b %y")}\n\t{nifty_p}\n\t{rf_p}\n\t{cash_p}''')

for date in date_lists[1:]:
    nifty_p.update(date=date)
    rf_p.update(date=date)
    # cash_p.deposit(date=date,amount=100)
    cash_p.withdraw(date=date,amount=7000)
    print(f'''{date.strftime("%d %b %y")}\n\t{nifty_p}\n\t{rf_p}\n\t{cash_p}''')

print(to_currency(rf.future_exit_value(investment_amount=10_000,
    entry_date=date_lists[0],exit_date=date_lists[2])))


Nifty rf
01 Feb 15
	Nifty_on_01Feb15 ₹ 9,980 | exit: ₹ 9,960, entry: ₹ 9,980 | update:01Feb15
	rf_on_01Feb15 ₹ 9,990 | exit: ₹ 9,970, entry: ₹ 9,990 | update:01Feb15
	cash_on_01Feb15 ₹ 10,000 | exit: ₹ 10,000, entry: ₹ 10,000 | update:01Feb15
03 Jul 15
	Nifty_on_01Feb15 ₹ 9,613 | exit: ₹ 9,593, entry: ₹ 9,980 | update:03Jul15
	rf_on_01Feb15 ₹ 10,235 | exit: ₹ 10,148, entry: ₹ 9,990 | update:03Jul15
	cash_on_01Feb15 ₹ 3,000 | exit: ₹ 3,000, entry: ₹ 10,000 | update:03Jul15
04 Feb 17
	Nifty_on_01Feb15 ₹ 9,903 | exit: ₹ 9,883, entry: ₹ 9,980 | update:04Feb17
	rf_on_01Feb15 ₹ 11,232 | exit: ₹ 10,845, entry: ₹ 9,990 | update:04Feb17
	cash_on_01Feb15 ₹ 3,000 | exit: ₹ 3,000, entry: ₹ 10,000 | update:04Feb17
07 Feb 22
	Nifty_on_01Feb15 ₹ 19,502 | exit: ₹ 18,532, entry: ₹ 9,980 | update:07Feb22
	rf_on_01Feb15 ₹ 15,040 | exit: ₹ 14,014, entry: ₹ 9,990 | update:07Feb22
	cash_on_01Feb15 ₹ 3,000 | exit: ₹ 3,000, entry: ₹ 10,000 | update:07Feb22
₹ 10,848


In [2]:
def response (a):
    if a==1:
        return 0
    else:
        return a

In [4]:
response(11)

11

In [15]:
a = [2,3,1,4,1,5]
new_a = []
for i in a:
    print(response(i))
    if response(i) != 0:
        new_a.append(i)
a = new_a
a

2
3
0
4
0
5


[2, 3, 4, 5]

In [10]:
a.pop(1)
a

[2, 1, 4, 1, 5]

In [12]:
for idx,i in enumerate(a):
    print(idx)

0
1
2
3
4


N_on_04Jan2022 @ 9301.588127097346, exit @ 9383.349908032744, bought @ 9980

In [ ]:


instruments = [{'name':'NIFTY 50','filename':'NIFTY50_Data.csv'},
                {'name':'NIFTY High Beta','filename':'NIFTY HIGH BETA 50_Data.csv'},
                {'name':'NIFTY 500','filename':'NIFTY 500_Data.csv'},
                {'name':'NIFTY BANK','filename':'NIFTY BANK_Data.csv'}]

for instrument in instruments:
    instrument['df'] = pd.read_csv(os.path.join(data_folder_name, instruments_folder_name, instrument['filename']))
    instrument['df']['Date'] = pd.to_datetime(instrument['df']['Date'])
    instrument['df'].sort_values(by='Date', inplace=True)
    instrument['df']['change'] = (instrument['df']['Close'] - instrument['df']['Close'].shift(periods=1)) / instrument['df']['Close'].shift(periods=1)
    print(f'''{instrument['name']}\t mu = {round(instrument['df']['change'].mean()*100,2)}%\t sigma = {round(instrument['df']['change'].std()*100,2)}%''')

np.random.seed(43)